In [1]:
import pandas as pd
import json
import csv
import re

# Importar librerías para automatización de navegadores web
# -----------------------------------------------------------------------
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys



# Importar librería para hacer solicitudes HTTP y extracción de datos
# -----------------------------------------------------------------------
import requests
from bs4 import BeautifulSoup


# Importar librería para suprimir advertencias
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')


C:\Users\Isabel\AppData\Local\Temp\ipykernel_12212\1604821139.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [19]:
d_drama = {}

In [17]:

def llamar_api(inicio, final, genero, d_salida):

	for year in range(inicio, (final+1)):
		por_anio = {f'{year}': []}

		for page in range(1,51):
			url = "https://moviesdatabase.p.rapidapi.com/titles"

			querystring = {"genre":f"{genero.capitalize()}","year":f"{year}","page":f"{page}"}

			headers = {
				"X-RapidAPI-Key": "902358e2e8msh3d16b2ef47d6b52p1fc5dajsnb0a5bd959614",
				"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
			}

			response = requests.get(url, headers=headers, params=querystring)
			peliculas = response.json()
			print(f'Año: {year}. Página: {page}. Estatus: {response.status_code}')

			for pelicula in peliculas['results']:
				por_anio[f'{year}'].append(pelicula)
	
		d_salida.update(por_anio)
		

In [64]:
llamar_api(2007, 20010, 'Drama', d_drama)

Año: 2007. Página: 1. Estatus: 200
Año: 2007. Página: 2. Estatus: 200
Año: 2007. Página: 3. Estatus: 200
Año: 2007. Página: 4. Estatus: 200
Año: 2007. Página: 5. Estatus: 200
Año: 2007. Página: 6. Estatus: 200
Año: 2007. Página: 7. Estatus: 200
Año: 2007. Página: 8. Estatus: 200
Año: 2007. Página: 9. Estatus: 200
Año: 2007. Página: 10. Estatus: 200
Año: 2007. Página: 11. Estatus: 200
Año: 2007. Página: 12. Estatus: 200
Año: 2007. Página: 13. Estatus: 200
Año: 2007. Página: 14. Estatus: 200
Año: 2007. Página: 15. Estatus: 200
Año: 2007. Página: 16. Estatus: 200
Año: 2007. Página: 17. Estatus: 200
Año: 2007. Página: 18. Estatus: 200
Año: 2007. Página: 19. Estatus: 200
Año: 2007. Página: 20. Estatus: 200
Año: 2007. Página: 21. Estatus: 200
Año: 2007. Página: 22. Estatus: 200
Año: 2007. Página: 23. Estatus: 200
Año: 2007. Página: 24. Estatus: 200
Año: 2007. Página: 25. Estatus: 200
Año: 2007. Página: 26. Estatus: 200
Año: 2007. Página: 27. Estatus: 200
Año: 2007. Página: 28. Estatus: 200
A

KeyError: 'results'

In [65]:
len(d_drama['2009'])

500

In [66]:
with open('ExtraccionDrama.json', 'w') as fichero:
    json.dump(d_drama, fichero)

In [30]:
def criba_generos(diccionario_origen, diccionario_salida, inicio, final):

    for year in range(inicio, (final+1)):
        limpio_por_year = {f'{year}': []}

        for i in diccionario_origen[f'{year}']:
            if i['titleType']['id'] == 'movie':
                limpio_por_year[f'{year}'].append(i)
            elif i['titleType']['id'] == 'short':
                limpio_por_year[f'{year}'].append(i)
            else:
                continue
        
        diccionario_salida.update(limpio_por_year)

In [25]:
d_limpio = {}

In [67]:
criba_generos(d_drama, d_limpio, 2007, 2009)

In [68]:
d_limpio.keys()

dict_keys(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009'])

In [69]:
with open('LimpioDrama.json', 'w') as fichero:
    json.dump(d_limpio, fichero)

In [7]:
def lista_tuplas(origen, inicio, final, genero):
    lista_datos = []
    for year in range(inicio, (final+1)):
        
        for i in origen[f'{year}']:
            tupla_pelicula = []
            try:
                tupla_pelicula.append(i['id'])
                tupla_pelicula.append(genero)
                tupla_pelicula.append(i["titleText"]['text'])
                tupla_pelicula.append(i['titleType']['id'])
                try:
                    tupla_pelicula.append(i['releaseDate']['year'])
                except:
                    tupla_pelicula.append('desconocido')
                try:
                    tupla_pelicula.append(i['releaseDate']['month'])
                except:
                    tupla_pelicula.append('desconocido')
                
            except:
                continue
                
        
            lista_datos.append(tuple(tupla_pelicula))

    for i in lista_datos:
        try:
            if i[3] < 2000:
                lista_datos.remove(i)
        except:
            continue

    return lista_datos

In [4]:
import json
with open('LimpioDrama.json', 'r') as fichero:
    drama = json.load(fichero)

In [8]:
pal_json = lista_tuplas(drama, 2000, 2009, 'drama')

In [ ]:
with open('JsonDrama.json', 'w') as fichero:
    json.dump(pal_json, fichero)

In [70]:
tuplas_drama = lista_tuplas(drama, 2000, 2009, 'drama')

In [1]:
tuplas_drama

NameError: name 'tuplas_drama' is not defined

In [72]:
df_drama = pd.DataFrame(tuplas_drama)

In [74]:
df_drama

,0,1,2,3,4,5
0,tt0102362,drama,Istota,movie,desconocido,desconocido
1,tt0113092,drama,For the Cause,movie,2002,5
2,tt0113086,drama,Florentino y el diablo,movie,2000,4
3,tt0116748,drama,Karobaar: The Business of Love,movie,2000,9
4,tt0115874,drama,Chateaubriand - Cabeça de Paraíba,short,2000,12
...,...,...,...,...,...,...
3965,tt1001521,drama,Layover,movie,2009,2
3966,tt1001508,drama,He's Just Not That Into You,movie,2009,4
3967,tt1002561,drama,2:13,movie,2009,4
3968,tt1002741,drama,Something Blue,movie,2009,9


In [73]:
df_drama.to_csv('TuplasDrama.csv', index=False)

In [8]:
lista_datos = lista_tuplas(drama, 2000, 2015)

In [12]:
with open('drama_2014_limpio.json', 'w') as doc:
    json.dump(lista_datos, doc)

In [10]:
df = pd.DataFrame(lista_datos)

In [11]:
df

,0,1,2,3,4
0,tt0113092,For the Cause,movie,2002,5.0
1,tt0116748,Karobaar: The Business of Love,movie,2000,9.0
2,tt0116391,Gang,movie,2000,4.0
3,tt0118694,Fa yeung nin wah,movie,2001,3.0
4,tt0118852,Chinese Coffee,movie,2000,9.0
...,...,...,...,...,...
2107,tt10033718,Though I Walk Through the Valley,short,2014,11.0
2108,tt10033820,The Soul of a Policeman,short,2014,5.0
2109,tt10036308,The Drive,short,2014,6.0
2110,tt10039008,Prathighatana,movie,2014,4.0


NUEVO INTENTO SELENIUM - IMDB

In [3]:
#with open('JsonDrama.json', 'r') as fichero:
  #  busqueda = json.load(fichero)

In [3]:
busqueda = ['tt0113092', 'tt0116748', 'tt0116391']

In [7]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(3)

# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(2)

# variables donde iremos almacenando todo
fallos = []

lista_directores = []

# entrando en cada título
for peli in busqueda:
    tupla_datos = [f'{peli}',]

    # id en buscador
    driver.find_element('css selector', '#suggestion-search').send_keys(peli, Keys.ENTER)
    sleep(1)

    # direccion
    directores = []
    for director in range(1,4):
        
        try:
            direccion = driver.find_element('xpath', f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[{director}]/a').text
            directores.append(direccion)
        except:
            continue

    lista_directores.append(directores)


        

Cagaste
Cagaste
Cagaste
Cagaste
Cagaste


In [8]:
lista_directores

[['David Douglas', 'Tim Douglas'], ['Rakesh Roshan'], ['Mazhar Khan']]

In [ ]:
'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[2]/a'

In [11]:
directores

['David Douglas']